In [1]:
# app/hear_utils.py

import librosa
import numpy as np
import torch
# from audio_utils import preprocess_audio  # From HeAR
from python.data_processing.audio_utils import preprocess_audio

SAMPLE_RATE = 16000
EXPECTED_SAMPLES = SAMPLE_RATE * 2

def load_and_prepare_audio(file_path):
    audio, sr = librosa.load(file_path, sr=SAMPLE_RATE, mono=True)
    if len(audio) < EXPECTED_SAMPLES:
        audio = np.pad(audio, (0, EXPECTED_SAMPLES - len(audio)))
    else:
        audio = audio[:EXPECTED_SAMPLES]
    return torch.tensor(audio, dtype=torch.float32).unsqueeze(0)

def get_hear_embedding(file_path):
    audio_tensor = load_and_prepare_audio(file_path)
    embedding = preprocess_audio(audio_tensor)  # [1, 1, 192, 128]
    return embedding.flatten().detach().numpy()  # [24576]


In [2]:
embedding = get_hear_embedding(r"H:\hustle\coding\Curebay_assignment\Asthma Detection Dataset Version 2\asthma\P1AsthmaIE_1.wav")


In [3]:
# embedding = get_hear_embedding("test.wav")
print(embedding.shape)  # (24576,)


(24576,)


In [4]:
# model/train_model.py

import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib
# from app.hear_utils import get_hear_embedding

DATA_DIR = r"H:\hustle\coding\Curebay_assignment\Asthma Detection Dataset Version 2"
X, y = [], []

for label in os.listdir(DATA_DIR):
    folder = os.path.join(DATA_DIR, label)
    if os.path.isdir(folder):
        for file in os.listdir(folder):
            if file.endswith(".wav"):
                try:
                    path = os.path.join(folder, file)
                    emb = get_hear_embedding(path)
                    X.append(emb)
                    y.append(label)
                except Exception as e:
                    print(f"Error: {file} - {e}")

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
joblib.dump(clf, "model/classifier.pkl")

print("\nClassification Report:")
print(classification_report(y_test, clf.predict(X_test)))



Classification Report:
              precision    recall  f1-score   support

   Bronchial       0.50      0.04      0.07        28
      asthma       0.75      0.91      0.83        57
        copd       0.77      0.80      0.79        82
     healthy       0.44      0.36      0.40        22
   pneumonia       0.68      0.85      0.75        54

    accuracy                           0.71       243
   macro avg       0.63      0.59      0.57       243
weighted avg       0.68      0.71      0.67       243

